<a href="https://colab.research.google.com/github/prakhar-chaurasiya/final_project_nism/blob/main/order_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

to do 


*   demand and supply
*   buy max price and sell min price per second



In [59]:
import pandas as pd
import numpy as np
from datetime import datetime
import tensorflow as tf
np.set_printoptions(suppress=True)

dlf_file = '/content/drive/MyDrive/Colab Notebooks/FA/HIGHFREQ-DATA/highfreq/dlf.order'
order_df = pd.read_csv(dlf_file, sep='|', header=None)
order_df.columns =['session', 'orderid', 'side', 'action', 'order_time', 'price', 'fquant', 'dquant', 'market', 'ioc', 'stoploss', 'triggerprice']
jiffy = 65536
ist = 330*60
order_df['time'] = order_df['order_time'].apply(lambda x: datetime.fromtimestamp(x/jiffy))#.strftime('%Y-%m-%d %H:%M:%S'))
# order_df['min'] = order_df['order_time'].apply(lambda x: datetime.fromtimestamp(x/jiffy).strftime('%H:%M:%S'))
order_df['sec'] = pd.to_datetime(order_df['time']).dt.strftime("%H%M%S").astype('int')
order_df

,session,orderid,side,action,order_time,price,fquant,dquant,market,ioc,stoploss,triggerprice,time,sec
0,PO,1000000000002582,B,1,84908467825804,27170,1,0,N,N,N,0,2011-01-21 09:00:00.174011,90000
1,PO,1000000000002729,S,1,84908467825877,28995,100,0,N,N,N,0,2011-01-21 09:00:00.175125,90000
2,PO,1000000000003157,S,1,84908467826858,29630,6,0,N,N,N,0,2011-01-21 09:00:00.190094,90000
3,PO,1000000000003255,S,1,84908467827494,30500,1000,0,N,N,N,0,2011-01-21 09:00:00.199799,90000
4,PO,1000000000003402,S,1,84908467828919,29070,500,0,N,N,N,0,2011-01-21 09:00:00.221542,90000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056884,RM,1000000001596191,S,3,84910125129363,29500,100,0,N,N,N,0,2011-01-21 16:01:28.619431,160128
1056885,RM,1000000008423767,S,3,84910125129364,29800,28,0,N,N,N,0,2011-01-21 16:01:28.619446,160128
1056886,RM,1000000001228775,B,3,84910125129365,28650,1,0,N,N,N,0,2011-01-21 16:01:28.619461,160128
1056887,RM,1000000009083263,S,3,84910125129366,29400,1,0,N,N,N,0,2011-01-21 16:01:28.619476,160128


order min max price of a second

In [ ]:
all_prices_np = order_df[['market', 'sec', 'price']].values
prices_np = all_prices_np[all_prices_np[:, 0] == 'N'][:, 1:]
prices_tf = tf.convert_to_tensor(prices_np, dtype=tf.int32)
rows, row_idx = np.unique(prices_np[:, 0], return_inverse=True)
ohlc = np.zeros((rows.shape[0], 5), dtype=np.float32)
for i, r in enumerate(rows):
    p = prices_tf[prices_tf[:, 0]==r][:, 1].numpy()
    o = p[0]
    h = p.max()
    l = p.min()
    c = p[-1]
    ohlc[i, 0] = r
    ohlc[i, 1] = o
    ohlc[i, 2] = h
    ohlc[i, 3] = l
    ohlc[i, 4] = c

In [56]:
ohlc

array([[ 90000,  27170,  31790,  27100,  28400],
       [ 90001,  28000,  31500,  26600,  26600],
       [ 90002,  30100,  31580,  26400,  29600],
       ...,
       [155753,  29300,  29300,  29300,  29300],
       [155957,  29600,  29600,  29600,  29600],
       [160128,  30000,  30000,  28200,  28200]], dtype=int32)

In [ ]:
# pure numpy takes too much of time to execute so use tensorflow
# ohlc1 = np.zeros((rows.shape[0], 5), dtype=np.int32)
# for i, r in enumerate(rows):
#     p = prices_np[prices_np[:, 0]==r][:, 1]
#     o = p[0]
#     h = p.max()
#     l = p.min()
#     c = p[-1]
#     ohlc1[i, 0] = r
#     ohlc1[i, 1] = o
#     ohlc1[i, 2] = h
#     ohlc1[i, 3] = l
#     ohlc1[i, 4] = c
#     print(r)


# Rough

In [18]:
order_df['sec'].unique()

array([ 90000,  90001,  90002, ..., 155902, 155957, 160128])

In [28]:
all_prices_np = order_df[['market', 'sec', 'price']].values
prices_np = all_prices_np[all_prices_np[:, 0] == 'N'][:, 1:]
rows, row_idx = np.unique(prices_np[:, 0], return_inverse=True)
# ohlc = np.zeros((rows.shape[0], 2), dtype=prices_np.dtype)
# for i, (r, id1, id2) in enumerate(zip(rows, np.unique(row_idx), np.unique(row_idx)[1:])):
#     ohlc[i, 0] = r
#     ohlc[i, 1] = prices_np[id1:id2, :]
# ohlc

In [ ]:
prices_tf = tf.convert_to_tensor(prices_np, dtype=tf.int32)
print(prices_tf)